In [ ]:
# Import Libraries
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Dataset
dataset = [
    "The sky is clear today.",
    "It looks like it's going to rain.",
    "The weather forecast predicts sunny skies tomorrow.",
    "I love walking in the rain.",
    "Rainy days make me feel peaceful.",
    "The sun is shining bright today.",
    "It's incredibly cold outside today.",
    "The wind is strong this afternoon.",
    "I forgot my umbrella, and now I'm soaking wet.",
    "The leaves turn beautiful colors in the fall.",
    "Winter is coming, and it's getting colder.",
    "The warmth of the sun feels so good on a cold day.",
    "Snow is falling outside.",
    "I hope it doesn't rain this weekend.",
    "The weather is perfect for a picnic.",
    "It's so hot outside that I can't think straight.",
    "A storm is coming this evening.",
    "The weather has been really unpredictable lately.",
    "I'm looking forward to the spring flowers.",
    "Autumn is my favorite season because of the cool weather.",
]


In [ ]:
# Preprocessing
tokenizer = Tokenizer()
tokenizer.fit_on_texts(dataset)
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in dataset:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Predictors and label
X, labels = input_sequences[:,:-1],input_sequences[:,-1]

In [ ]:
# Model Definition
from tensorflow.keras.layers import Dropout

model = Sequential()
model.add(Embedding(total_words, 50, input_length=max_sequence_len-1))
model.add(LSTM(150, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.01), metrics=['accuracy'])

In [ ]:
# Training
model.fit(X, labels, epochs=100, verbose=1)

Epoch 1/100
4/4 [==============================] - 5s 41ms/step - loss: 4.5330 - accuracy: 0.0413
Epoch 2/100
4/4 [==============================] - 0s 36ms/step - loss: 4.5202 - accuracy: 0.0661
Epoch 3/100
4/4 [==============================] - 0s 41ms/step - loss: 4.4997 - accuracy: 0.0661
Epoch 4/100
4/4 [==============================] - 0s 42ms/step - loss: 4.4574 - accuracy: 0.0661
Epoch 5/100
4/4 [==============================] - 0s 40ms/step - loss: 4.3775 - accuracy: 0.0661
Epoch 6/100
4/4 [==============================] - 0s 41ms/step - loss: 4.3720 - accuracy: 0.0661
Epoch 7/100
4/4 [==============================] - 0s 42ms/step - loss: 4.3449 - accuracy: 0.0661
Epoch 8/100
4/4 [==============================] - 0s 38ms/step - loss: 4.3345 - accuracy: 0.0661
Epoch 9/100
4/4 [==============================] - 0s 37ms/step - loss: 4.3179 - accuracy: 0.0661
Epoch 10/100
4/4 [==============================] - 0s 38ms/step - loss: 4.3121 - accuracy: 0.0661
Epoch 11/100
4/4 [=

In [ ]:
seed_text = "The weather today"
next_words = 5

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predictions = model.predict(token_list, verbose=0)
    predicted = np.argmax(predictions, axis=-1)[0]
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

The weather today perfect for a picnic good
